In [ ]:
"""
Earthquake Prediction Inference Script Using CNN Model on PSD Vectors
======================================================================

This script loads a trained 1D CNN model and applies it to a time range of 
Power Spectral Density (PSD) feature vectors, making predictions for 
earthquake vs background classification.

Main Steps:
-----------
1. Load pre-computed normalization statistics (mean and std) from `.npy` files.
2. Load a trained CNN model (PyTorch) for earthquake detection.
3. Define a test time range and query PSD vectors using `psd_vectors_from_range`.
4. Run inference on each PSD vector to compute earthquake/background probabilities.
5. Save all predictions into three CSV files:
    - Full log of all predictions (`Earthquake_Predictions_5_05_2025.csv`)
    - Filtered log of only predicted earthquakes (`Earthquake_Event_Log_5_05_2025.csv`)
    - Filtered log of high-confidence earthquakes (prob ≥ 0.90) 
      (`Earthquake_Strong_Event_Log_5_05_2025.csv`)

Dependencies:
-------------
- NumPy
- PyTorch
- CSV
- datetime
- os
- Local modules: `cnn_model.py`, `DataQueryUtils.py`

Notes:
------
- The CNN model expects PSD input of shape (1, 572).
- This script can be adapted for live monitoring or historical batch inference.
- Adjust the `start_time`, `end_time`, and model path as needed.

Ethan Gelfand, 08/06/2025

"""

import numpy as np
import torch
import os
import csv
from tqdm import tqdm
from datetime import datetime, timedelta, UTC
from cnn_model import EarthquakeCNN
from DataQueryUtils import psd_vectors_from_range 

# Load normalization stats
mean = np.load("../DataCollection_Preprocessing/Exported_Paros_Data/mean.npy")
std = np.load("../DataCollection_Preprocessing/Exported_Paros_Data/std.npy")

# Load trained model
model = EarthquakeCNN(input_length=572)
model.load_state_dict(torch.load("../ModelTraining/fold_outputs/fold_3/CNNmodel.pth", map_location="cpu"))
model.eval()
model.to("cpu")

# Set test time range (change as needed)
start_time = datetime(2025, 5, 5, 0, 0, 0, tzinfo=None)
end_time = datetime(2025, 5, 5, 23, 59, 59, tzinfo=None)
# Get all PSD vectors for that range
results = psd_vectors_from_range(
    start_time=start_time,
    end_time=end_time,
    sensor_id="141929",
    box_id="parost2",
    password="*****", # Replace with actual password
    fs_in=20,
    fs_out=100,
    window_duration=10,
    overlap=0.5,
    mean=mean,
    std=std
)


# Output CSV for all predictions
dir = "LoggedData"
os.makedirs(dir, exist_ok=True)
log_path = "LoggedData/Earthquake_Predictions_5_05_2025.csv"
event_log_path = "LoggedData/Earthquake_Event_Log_5_05_2025.csv"
strong_event_log_path = "LoggedData/Earthquake_Strong_Event_Log_5_05_2025.csv"

with open(log_path, mode="w", newline="") as f_all, open(event_log_path, mode="w", newline="") as f_event, open(strong_event_log_path, mode="w", newline="") as f_strong_event:
    writer_all = csv.writer(f_all)
    writer_event = csv.writer(f_event)
    writer_strong_event = csv.writer(f_strong_event)
    # Write headers
    header = ["query_time", "window_start", "window_end", "predicted_class", "prob_earthquake", "prob_background"]
    writer_all.writerow(header)
    writer_event.writerow(header)
    writer_strong_event.writerow(header)

    for (window_start, window_end, psd_vector) in tqdm(results, desc="Running inferences", colour="green"):
        input_tensor = torch.tensor(psd_vector, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1).numpy().flatten()
            pred = np.argmax(probs)

        row = [
            datetime.now(UTC).isoformat(timespec="seconds"),
            window_start.isoformat(timespec="seconds"),
            window_end.isoformat(timespec="seconds"),
            int(pred),
            round(float(probs[1]), 5),
            round(float(probs[0]), 5)
        ]

        writer_all.writerow(row)
        if pred == 1:
            writer_event.writerow(row)
            if probs[1] >= 0.90:
                writer_strong_event.writerow(row)

print("Completed Inferences")



Running inferences: 100%|██████████| 1439/1439 [00:00<00:00, 1967.62it/s]

Completed Inferences
